#### Проверка доступности CUDA & cuDNN

In [1]:
!python scripts/check-cuda.py

TensorFlow version:  2.13.1
Keras version:  2.13.1
CUDA is not available. TensorFlow does not have CUDA support.


#### Cоздает и обучает нейронную сеть для классификации рукописных цифр из набора данных MNIST с использованием TensorFlow и Keras для построения последовательной модели.

In [2]:
!python scripts/mnist-test.py

Epoch 1/5

1875/1875 [==============================] - 4s 2ms/step - loss: 0.2973 - accuracy: 0.9138
Epoch 2/5

1875/1875 [==============================] - 3s 2ms/step - loss: 0.1450 - accuracy: 0.9574
Epoch 3/5

1875/1875 [==============================] - 3s 2ms/step - loss: 0.1082 - accuracy: 0.9672
Epoch 4/5

1875/1875 [==============================] - 4s 2ms/step - loss: 0.0888 - accuracy: 0.9728
Epoch 5/5

1875/1875 [==============================] - 3s 2ms/step - loss: 0.0773 - accuracy: 0.9753


2024-01-07 20:40:03.575085: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE SSE2 SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# 1. Подготовка

## 1.1 Установка зависимостей

In [ ]:
!pip install tf-nightly[and-cuda] keras opencv-python matplotlib

## 1.2 Импортирование стандартных зависимостей 

In [7]:
import cv2
import os
import random
import numpy as np
from matplotlib import pyplot as plt

'4.8.1'

## 1.3 Импортирование зависимостей TensorFlow (Функциональный API)

In [2]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
import tensorflow as tf

## 1.4 Установка роста потребления памяти GPU для предотвращения  ошибо OOM.

In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
    print(gpu)

## 1.5 Создание структуры директорий

#### Установка путей

In [11]:
POS_PATH = os.path.join('data', 'positive')
NEG_PATH = os.path.join('data', 'negative')
ANC_PATH = os.path.join('data', 'anchor')

INPUT_IMG_PATH = os.path.join('application_data', 'input_image')
VERIF_IMG_PATH = os.path.join('application_data', 'verification_images')

DIRECTORIES = [POS_PATH, NEG_PATH, ANC_PATH, INPUT_IMG_PATH, VERIF_IMG_PATH]

#### Создание директорий

In [16]:
for dir in DIRECTORIES:
    try:
        os.makedirs(dir)
    except FileExistsError:
        print(f"Директория {dir} уже существует")

Директория data\positive уже существует
Директория data\negative уже существует
Директория data\anchor уже существует
Директория application_data\input_image уже существует
Директория application_data\verification_images уже существует


# 2 Сбор данных

Скачать все изображения в виде gzip-tar-файла  https://vis-www.cs.umass.edu/lfw/#download

## 2.1 Распаковка набора данных

In [22]:
!tar -xf lfw.tgz 

In [23]:
!rm lfw.tgz

## 2.2 Перемещение изображений в директорию data/negative

In [28]:
for directory in os.listdir('lfw'):
    for file in os.listdir(os.path.join('lfw', directory)):
        EX_PATH = os.path.join('lfw', directory, file)
        NEW_PATH = os.path.join(NEG_PATH, file)
        os.replace(EX_PATH, NEW_PATH)

In [30]:
!rm -r lfw

## 2.3 Сбор позитивных и якорных изображений

#### Библиотека для создания уникальных названий изображений.

In [5]:
import uuid

In [9]:
# Установка соединения с веб-камерой
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()

    # Обрезание кадра до 250x250px
    frame = frame[115:115+250,195:195+250, :]

    # Сбор якорных сэмплов
    if cv2.waitKey(1) & 0XFF == ord('a'):
        # Создание уникального названия изображения
        imgname = os.path.join(ANC_PATH, '{}.jpg'.format(uuid.uuid1()))
        # Запись изображения
        cv2.imwrite(imgname, frame)

    # Сбор позитивных сэмплов
    if cv2.waitKey(1) & 0XFF == ord('p'):
        # Создание уникального названия изображения
        imgname = os.path.join(POS_PATH, '{}.jpg'.format(uuid.uuid1()))
        # Запись изображения
        cv2.imwrite(imgname, frame)

    cv2.imshow('Image Collection', frame)
    if cv2.waitKey(1) & 0XFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

## 2.4 Дополнение данных

In [ ]:
def data_aug(img):
    data = []
    for i in range(5):
        img = tf.image.stateless_random_brightness(img, max_delta=0.02, seed=(1,2))
        img = tf.image.stateless_random_contrast(img, lower=0.6, upper=1, seed=(1,3))
        img = tf.image.stateless_random_flip_left_right(img, seed=(np.random.randint(100),np.random.randint(100)))
        img = tf.image.stateless_random_jpeg_quality(img, min_jpeg_quality=90, max_jpeg_quality=100, seed=(np.random.randint(100),np.random.randint(100)))
        img = tf.image.stateless_random_saturation(img, lower=0.9,upper=1, seed=(np.random.randint(100),np.random.randint(100)))
        data.append(img)
    return data

In [ ]:
img_path = os.path.join(ANC_PATH, '998b7e72-a950-11ee-ae26-00155d1571c4.jpg')
img = cv2.imread(img_path)
augmented_images = data_aug(img)

for image in augmented_images:
    cv2.imwrite(os.path.join(ANC_PATH, '{}.jpg'.format(uuid.uuid1())), image.numpy())

In [ ]:
for file_name in os.listdir(os.path.join(POS_PATH)):
    img_path = os.path.join(POS_PATH, file_name)
    img = cv2.imread(img_path)
    augmented_images = data_aug(img)

    for image in augmented_images:
        cv2.imwrite(os.path.join(POS_PATH, '{}.jpg'.format(uuid.uuid1())), image.numpy())

# 3 Загрузка и предобработка изображений

## 3.1 Получение каталогов изображений

In [16]:
anchor = tf.data.Dataset.list_files(os.path.join(ANC_PATH,'*.jpg')).take(300)
positive = tf.data.Dataset.list_files(os.path.join(POS_PATH,'*.jpg')).take(300)
negative = tf.data.Dataset.list_files(os.path.join(NEG_PATH,'*.jpg')).take(13233)

## 3.2 Предобработка - масштабирование и изменение размера

In [32]:
def preprocess(file_path):

    # Чтение изображения
    byte_img = tf.io.read_file(file_path)
    # Загрузка изображения
    img = tf.io.decode_jpeg(byte_img)
    
    # Изменение размера изображения на 105x105
    img = tf.image.resize(img, (105,105))
    # Масштабирование изображения в диапазоне от 0 до 1
    img = img / 255.0
    return img

## 3.3 Создание помеченного набора данных

In [35]:
positives = tf.data.Dataset.zip((anchor, positive, tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor)))))
negatives = tf.data.Dataset.zip((anchor, negative, tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchor)))))
data = positives.concatenate(negatives)

## 3.4 Сборка тренировочных и тестовых данных

In [48]:
def preprocess_twin(input_img, validation_img, label):
    return(preprocess(input_img), preprocess(validation_img), label)

#### Создание загрузчика данных

In [64]:
data = data.map(preprocess_twin)
data = data.cache()
data = data.shuffle(buffer_size=10000)

#### Данные для обучения

In [ ]:
train_data = data.take(round(len(data)*.7))
train_data = train_data.batch(16)
train_data = train_data.prefetch(8)

#### Данные для тестирования

In [ ]:
# Testing partition
test_data = data.skip(round(len(data)*.7))
test_data = test_data.take(round(len(data)*.3))
test_data = test_data.batch(16)
test_data = test_data.prefetch(8)